In [ ]:
import requests as rq # 网络请求基本库
from bs4 import BeautifulSoup # 清洗网页数据的库
import re # 正则表达式的库
    
# 在新笔趣阁，爬一篇名叫《临高启明》的小说第一章
if __name__ == '__main__':
    target = "https://www.xbiquge.la/0/858/708546.html" # 网址
    req = rq.get(url = target) # 用requests获取网页
    req.encoding = 'utf-8'# 设置网页编码
    html = req.text # 网页的html文件
    # print(html)
    con = BeautifulSoup(html,'lxml') # 利用soup库转换
    res = con.find('div',id='content') # 进行数据筛选
    # for p in res:
    #     print(p)
    res = re.split('(\xa0\xa0\xa0\xa0|\r)',res.text) # 分割文本
    ans=[]
    for x in res: # 筛选文本
        if(x!='\r' and x!='\xa0'*4 and x!='' and x!=' '):
            ans.append(x)
    print(ans)

In [3]:

import time
import requests
import os
import random
from lxml import etree
import webbrowser
header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.128 Safari/537.36 Edg/89.0.774.77"
}
noName = ['#','/','\\',':','*','?','\"','<','>','|']     #\/:*?"<>|
filePath = './保存小说'
def strZ(_str): #将特殊字符转换为空格
    ret = ''
    for _ in _str:
        if _ in noName:
            ret += " "
        else:
            ret += _
    return ret
def main():
    webbrowser.open('https://www.biquwx.la/')
    if not os.path.exists(filePath):
        os.mkdir(filePath)
    print('1.爬取指定小说')
    print('2.爬取整个站点')
    if input('使用哪种方式爬取小说？  ') == '1':
        appintDown()
    else :
        allDown()
    input("按下任意键退出")
def appintDown(): #爬取指定小说  前提是网页没错
    page_url = input('输入要爬取的小说网站(例如 [url=https://www.biquwx.la/10_10240/]https://www.biquwx.la/10_10240/[/url]) ：  ')
    page = requests.get(url=page_url, headers=header)
    if page.status_code == 200:  # 响应就爬取
        page.encoding = 'utf-8'
        page_tree = etree.HTML(page.text)
        page_title = page_tree.xpath('//div[@id="info"]/h1/text()')[0]
        _filePath = filePath + '/' + page_title
        if not os.path.exists(_filePath):
            os.mkdir(_filePath)
        page_dl_list = page_tree.xpath('//div[@class="box_con"]/div[@id="list"]/dl/dd')
        for _ in page_dl_list:
            _page_url = page_url + _.xpath('./a/@href')[0]
            _page_title = _filePath + '/' + strZ(_.xpath('./a/@title')[0]) + '.txt'
            _page = requests.get(_page_url, headers=header)
            if _page.status_code == 200:
                _page.encoding = 'utf-8'
                _tree = etree.HTML(_page.text)
                _page_content = _tree.xpath('//div[@id="content"]/text()')
                fileContent = ''
                for _ in _page_content:
                    fileContent += _ + '\n'
                with open(_page_title, 'w', encoding='utf-8') as fp:
                    fp.write(fileContent)
                    print('%s成功下载到本地' % (_page_title))
                time.sleep(random.uniform(0.05, 0.2))
def allDown(): #整个站点小说爬取
    url = 'https://www.biquge.info/wanjiexiaoshuo/'  # 目录
    page = requests.get(url=url, headers=header)
    if page.status_code == 200:  # 响应就爬取
        page.encoding = 'utf-8'
        tree = etree.HTML(page.text)
        page_last = tree.xpath('//div[@class="pagelink"]/a[@class="last"]/text()')[0]
        for page_i in range(1, int(page_last)):  # 小说页数遍历
            url = 'https://www.biquge.info/wanjiexiaoshuo/' + str(page_i)
            page = requests.get(url=url, headers=header)
            if page.status_code == 200:  # 响应就爬取
                page.encoding = 'utf-8'
                tree = etree.HTML(page.text)
                li_list = tree.xpath('//div[@class="novelslistss"]/ul/li')
                for li in li_list:
                    page_url = li.xpath('./span[@class="s2"]/a/@href')[0]  # 目录链接
                    page_title = strZ(li.xpath('./span[@class="s2"]/a/text()')[0])
                    page = requests.get(url=page_url, headers=header)
                    if page.status_code == 200:  # 响应就爬取
                        page.encoding = 'utf-8'
                        page_tree = etree.HTML(page.text)
                        _filePath = filePath + '/' + page_title
                        if not os.path.exists(_filePath):
                            os.mkdir(_filePath)
                        page_dl_list = page_tree.xpath('//div[@class="box_con"]/div[@id="list"]/dl/dd')
                        for _ in page_dl_list:
                            _page_url = page_url + _.xpath('./a/@href')[0]
                            _page_title = _filePath + '/' + strZ(_.xpath('./a/@title')[0]) + '.txt'
                            _page = requests.get(_page_url, headers=header)
                            if _page.status_code == 200:
                                _page.encoding = 'utf-8'
                                _tree = etree.HTML(_page.text)
                                _page_content = _tree.xpath('//div[@id="content"]/text()')
                                fileContent = ''
                                for _ in _page_content:
                                    fileContent += _ + '\n'
                                with open(_page_title, 'w', encoding='utf-8') as fp:
                                    fp.write(fileContent)
                                    print('%s成功下载到本地' % (_page_title))
                                time.sleep(random.uniform(0.05, 0.2))
if __name__ == '__main__':
    main()

1.爬取指定小说
2.爬取整个站点
./保存小说/绝世唐门/一份关于斗罗大陆II的访谈.txt成功下载到本地
./保存小说/绝世唐门/引子 神界！唐三一家.txt成功下载到本地
./保存小说/绝世唐门/第一章 灵眸少年（一）.txt成功下载到本地
./保存小说/绝世唐门/第一章 灵眸少年（二）.txt成功下载到本地
./保存小说/绝世唐门/第一章 灵眸少年（三）.txt成功下载到本地
./保存小说/绝世唐门/第一章 灵眸少年（四）.txt成功下载到本地
./保存小说/绝世唐门/第二章 天梦冰蚕（一）.txt成功下载到本地
./保存小说/绝世唐门/第二章 天梦冰蚕（二）.txt成功下载到本地
./保存小说/绝世唐门/第二章 天梦冰蚕（三）.txt成功下载到本地
./保存小说/绝世唐门/第二章 天梦冰蚕（四）.txt成功下载到本地
./保存小说/绝世唐门/第三章 百万年魂环（一）.txt成功下载到本地
./保存小说/绝世唐门/第三章.txt成功下载到本地
./保存小说/绝世唐门/第三章 百万年魂环（三）.txt成功下载到本地
./保存小说/绝世唐门/第三章 百万年魂环（四）.txt成功下载到本地
./保存小说/绝世唐门/第四章 霹雳贝贝（一）.txt成功下载到本地
./保存小说/绝世唐门/第四章 霹雳贝贝（二）.txt成功下载到本地
./保存小说/绝世唐门/第四章 霹雳贝贝（三）.txt成功下载到本地
./保存小说/绝世唐门/第四章 霹雳贝贝（四）.txt成功下载到本地
./保存小说/绝世唐门/第五章 初窥门径（一）.txt成功下载到本地
./保存小说/绝世唐门/第五章 初窥门径（二）.txt成功下载到本地
./保存小说/绝世唐门/第五章 初窥门径（三）.txt成功下载到本地
./保存小说/绝世唐门/第五章 初窥门径（四）.txt成功下载到本地
./保存小说/绝世唐门/第六章 斗罗大陆第一学院（一）.txt成功下载到本地
./保存小说/绝世唐门/第六章 斗罗大陆第一学院（二）.txt成功下载到本地
./保存小说/绝世唐门/第六章 斗罗大陆第一学院（三）.txt成功下载到本地
./保存小说/绝世唐门/第六章 斗罗大陆第一学院（四）.txt成功下载到本地
./保存小说/绝世唐门/第七章 新生入学（上）.txt成功下载到本地
./保存小说/绝世唐门/第七章 新生入学（上）.t

KeyboardInterrupt: 

In [ ]:
# 更简单的一个例子
import requests as rq
if __name__ == '__main__':
    website = 'https://www.baidu.com'
    req1 = rq.get(url = website)
    req1.encoding = 'utf-8'
    print(req1.text)
